In [62]:
import numpy as np
import pandas as pd
import scipy as sp
import resampy as rs 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa as tsa 
import statsmodels.formula.api as smf
import os
import glob
import random
import string

In [9]:
study_directory = '/Users/alexastefanko/lab/FoGdetection/Subjects'
output_directory = '/Users/alexastefanko/lab/FoGdetection/Output/'
sample_rate = 128

r_acc_ap_col = "R_acc"; #Column name for right accelerometer anterior-posterior
l_acc_ap_col = "L_acc"; #Column name for left accelerometer anterior-posterior
r_gyr_ml_col = "R_gyr"; #Column name for right gyroscope mediolateral
l_gyr_ml_col = "L_gyr"; #Column name for left gyroscop mediolateral

In [23]:
class Fog:
    def __init__(self, filename):
        self.filename = filename
        self.very_short_fog = None
        self.short_fog = None
        self.long_fog = None
        self.very_long_fog = None
        self.NN = None
        self.MM = None

In [89]:
def get_csv_files(directory):
    for dir_name, subdir_list, file_list in os.walk(directory):
        return [os.path.join(dir_name, name) for name in file_list if name.endswith(".csv")]
    
def correlation_based_method(data, filtered_data):
    r_ml = data[0]
    mean_r = np.mean(r_ml)
    r_for_cov = [x - mean_r for x in r_ml]
    
    l_ml = data[1]
    mean_l = np.mean(l_ml)
    l_for_cov = [x - mean_l for x in l_ml]

    num_points = len(r_ml)
    array = np.array([r_ml, l_ml])
        
    #R and L angular velocities cross-correlation to find a delay between two
    cross_cov = np.correlate(r_for_cov, l_for_cov, "full")
    shifter = 0
    for idx, num in enumerate(cross_cov):
        if num == np.max(cross_cov):
            shifter = idx
    
    starter = round(abs(shifter - (num_points/2)))
#     print(starter)
    
    #Sync R and L after correction for delay
    t_sensor_1 = filtered_data[0][starter:]
#     print(t_sensor_1)
    t_sensor_2 = filtered_data[1][:len(filtered_data) - starter]
    

In [60]:
subject_files = get_csv_files(study_directory)
dataframes = [pd.read_csv(file) for file in subject_files]

In [90]:
new_sample_rate = 200

x = 0
fogs = []
for df in dataframes:
    r_acc_ap = df[r_acc_ap_col].values
    l_acc_ap = df[l_acc_ap_col].values
    r_gyr_ml = df[r_gyr_ml_col].values
    l_gyr_ml = df[l_gyr_ml_col].values
    
    #Resample the data to 200 
    r_ml = rs.resample(r_gyr_ml, sample_rate, new_sample_rate)
    l_ml = rs.resample(l_gyr_ml, sample_rate, new_sample_rate)
    
    #apply low-pass butterworth filter
    a,b = sp.signal.butter(4,5/(200/2))
    sensor_data_filtered=sp.signal.filtfilt(a,b,[r_ml, l_ml]);

    correlation_based_method([r_ml, l_ml], sensor_data_filtered)
    
    fog = Fog(subject_files[x])
    fogs.append(fog)
    x = x + 1

17650
[0.42076951 0.33642017 0.22969082 ... 0.27793128 0.2427529  0.21333588]
17650
[0.42076951 0.33642017 0.22969082 ... 0.27793128 0.2427529  0.21333588]
